
# ViEWS 3 constituent models 

## ViEWS production system, pgm level


This notebook trains a set of regression models for use in the monthly updated ViEWS predicting fatalities ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing models

In [1]:
! viewser config list


  |:---------------------------------|:-----------------------------------------------------------------------------------------|
  | RETRY_FREQUENCY                  | 5                                                                                        |
  | LOG_LEVEL                        | INFO                                                                                     |
  | HANDSHAKE_PATH                   |                                                                                          |
  | REPO_URL                         | https://www.github.com/prio-data/viewser                                                 |
  | LATEST_KNOWN_VERSION             | 0.0.0                                                                                    |
  | NOTEBOOK_SERVER_IMAGE_REPOSITORY | prio-data/viewserspace                                                                   |
  | NOTEBOOK_SERVER_IMAGE_REGISTRY   | viewsregistry.azurecr.io                          

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from Predicting Fatalies repository

import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble,data_integrity_check
from ViewsEstimators import *

## Common parameters

In [4]:
# Common parameters:
dev_id = 'Fatalities003'
run_id = dev_id

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='pgm_level_fatalities',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

depvar="ln_ged_sb_dep"

RerunQuerysets = True
        
FutureStart = 518
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,408),"predict":(409,456)}
test_partitioner_dict = {"train":(121,456),"predict":(457,504)}
future_partitioner_dict = {"train":(121,504),"predict":(505,516)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS'
print('Setting Mydropbox to',Mydropbox)

Setting Mydropbox to /Users/havardhegre1/Dropbox (ViEWS)/ViEWS


## Retrieve data

In [5]:
# Create Markdown documentation of all querysets used
level = 'pgm'
qslist = ReturnQsList(level)
document_queryset(qslist,dev_id)

 .    
A dataset with 8 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 19 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 29 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 24 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 23 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 30 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 8 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 11 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 36 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 31 columns, with data between t 1 and 852. (13110 units)
 .    
A dataset with 42 columns, with data between t 1 and 852. (13110 units)
 .      o      O      O      o       .      o      O      O      o       .      o      O      O      o       .      o      O      O  

In [ ]:
! viewser tables show acled2_pgm

In [ ]:
#if RerunQuerysets:
#    import pgm_querysets

In [6]:
from FetchData import fetch_pgm_data_from_model_def

Datasets=fetch_pgm_data_from_model_def(qslist)

 .    baseline: A dataset with 8 columns, with data between t = 1 and 852; 13110 units.
 .    conflicttreelag: A dataset with 8 columns, with data between t = 1 and 852; 13110 units.
 .    broad: A dataset with 23 columns, with data between t = 1 and 852; 13110 units.
 .    conflictlong: A dataset with 19 columns, with data between t = 1 and 852; 13110 units.
 .    conflicthist: A dataset with 30 columns, with data between t = 1 and 852; 13110 units.
 .    natsoc: A dataset with 24 columns, with data between t = 1 and 852; 13110 units.
 .    conflictsptime_dist: A dataset with 11 columns, with data between t = 1 and 852; 13110 units.
 .    escwa_drought: A dataset with 29 columns, with data between t = 1 and 852; 13110 units.


In [ ]:
for ds in Datasets:

    data_integrity_check(ds,depvar)

# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

In [ ]:
dev_id

# Specify models in ensemble

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('pgm')
    

for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

In [ ]:
document_ensemble(ModelList,'sb')

In [ ]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'pgm'
includeFuture = False
force_rewrite = True
force_retrain = True
store_remote = False

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList[10:]:
    modelstore = storage.Storage()
    ct = datetime.now()
    print(i, model['modelname'])
    print('Calibration partition', ct)
    model['Algorithm_text'] = str(model['algorithm'])
    model['RunResult_calib'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "calib",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_calib',
            author_name        = "JED",
    )

    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
    ct = datetime.now()
    if force_rewrite:
        print(model['predstore_calib'], ', run',  run_id, 'force_rewrite=True, predicting')
        predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)

        predictions_calib.to_parquet(model['predstore_calib']+'.parquet')
        if store_remote:
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'],overwrite=True)
    else:
        print('Trying to retrieve predictions', ct)
        try:
            predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
        except KeyError:
            print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'])
                
    ct = datetime.now()
    print('Test partition', ct)
    modelstore = storage.Storage()
    model['RunResult_test'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"test":test_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "test",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_test',
            author_name        = "JED",
    )
    ct = datetime.now()
    if force_rewrite:
        print(model['predstore_test'], ', run',  run_id, 'force_rewrite=True, predicting')
        predictions_test = model['RunResult_test'].run.predict("test","predict", model['RunResult_test'].data)
        
        predictions_test.to_parquet(model['predstore_test']+'.parquet')
        if store_remote:
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'],overwrite=True)
    else:
        print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
        try:
            predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
        except KeyError:
            print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'])
    # Predictions for true future
    if includeFuture:
        ct = datetime.now()
        print('Future', ct)
        modelstore = storage.Storage()
        model['RunResult_future'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_future',
                author_name        = "JED",
        )
        ct = datetime.now()
        if force_rewrite:
            print(model['predstore_future'], ', run',  run_id, 'force_rewrite=True, predicting')
            predictions_future = model['RunResult_future'].run.predict(FutureStart, model['RunResult_future'].data)
            predictions_future.to_parquet(model['predstore_future']+'.parquet')

            if store_remote:
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'],overwrite=True)
        else:
            print('Trying to retrieve predictions', ct)
            model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
            try:
                predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
            except KeyError:
                print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
                predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
#    model['algorithm'] = []
    i = i + 1

print('All done')

In [ ]:
ModelMetaData = pd.DataFrame(ModelList)

In [ ]:
ModelMetaData